In [1]:
#get dataframe of tweet_id and user_id
import pandas
import csv
from twython import Twython

df = pandas.read_csv('gamergate_tweets.csv')
user_ids = df['user_id'].tolist()[:5000]

# Consumer & Auth keys and tokens for authentication
CONSUMER_KEY = "48r56upQvsnTiSDfY4CNwIpCQ"
CONSUMER_SECRET = "VqZhFW7GqogdKXJnpuS0umVe211bFgoCdaQlRdRMayvrbdDh3k"
OAUTH_TOKEN = "18000528-Scl8a1pLinqP4KWtaa3f8EN0M1vqAGTyYXMtsLIGm"
OAUTH_TOKEN_SECRET = "NcIvLeqVbwwBegabvo7946FVIaDN0CAaJ7aS2aV7pZFzC"

twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)


In [2]:
# GET followers/list
def get_follower_list(twitter, user_id):
    followers = []
    cursored = twitter.cursor(twitter.get_followers_ids, id=user_id)
    for follower in cursored:
        followers.append(follower)
    return followers

# GET users/lookup
def get_user_lookup(twitter, current):
    user_obj = twitter.lookup_user(user_id=current)
    return user_obj
    
# get rate limits
# status = twitter.get_application_rate_limit_status()
# print status["resources"]["users"]['/users/lookup']

user_list = []
start = 0
end = 100
while end <= 5000:
    current = user_ids[start:end]
    start += 100
    end += 100
    users = get_user_lookup(twitter, current)
    for i in range(0, len(users)):
        each_user = []
        each_user.append(user_ids[i]) # user[0] = user_id
        each_user.append(users[i]['followers_count']) # user[1] = follower count
        each_user.append(users[i]["favourites_count"])
        user_list.append(each_user)

In [3]:
users_df = pandas.DataFrame(user_list, columns=["User ID", "Follower Count", "Favourites Count"])
                
print users_df.sort_values(["Follower Count", "Favourites Count",], ascending=[False, False])

         User ID  Follower Count  Favourites Count
1997  1004731904          224069              6977
328     26258609          104852             43600
777   2774051122           86819              1586
549   2423739066           86565             11490
1556    22655375           86565             11490
189     26258609           68680             25909
1358  1004731904           68680             25909
2710   992871366           68680             25909
3557   262538589           68680             25909
229   2756873076           57753              1310
1388   162572003           57405              2809
2084   775543549           55073              2634
2096  2772953185           55073              2634
2492   992871366           55073              2634
2935   142413785           52133             17279
2460  2549103608           39487             31725
3392  2826306403           39487             31725
736    171848975           34020             39777
2567   992871366           3299